In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import ElementClickInterceptedException, JavascriptException, NoSuchElementException
import time
from datetime import date, datetime, timedelta 
import numpy as np
import pandas as pd



### Waits for the Pick-Up Location inside the search-widget-form to load

In [90]:

# fixes deprecation error https://stackoverflow.com/questions/64717302/deprecationwarning-executable-path-has-been-deprecated-selenium-python

s=Service(ChromeDriverManager().install())
browser = webdriver.Chrome(service=s)
#browser = webdriver.Chrome(executable_path = "C:\Program Files\Chromedriver\chromedriver.exe")
url = 'https://www.campertravelusa.com/'
browser.get(url)



# waits for page elements to load https://stackoverflow.com/questions/26566799/wait-until-page-is-loaded-with-selenium-webdriver-for-python
delay = 3 # seconds
try:
    myElem = WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.XPATH,"//input[contains(@id, 'pickupLocation')]")))
    print ("Page is ready!")
except TimeoutException:
    print ("Loading took too much time!")



#browser.find_element(By.XPATH,"//input[contains(@id, 'pickupLocation')]").send_keys('Orlando')

#//*[@id="search_name"]

#for city in cities:
#browser.find_element(By.XPATH,"//input[contains(@id, 'pickupLocation')]").click()#send_keys(city['city'])
    #browser.find_element(By.XPATH,f"//div[@class = 'autocomplete-suggestion'][@data-val = '{city['city']}']").click()


[WDM] - Downloading: 100%|██████████| 6.21M/6.21M [00:01<00:00, 5.11MB/s]


Page is ready!


### Step 1: Select Pick-up Location and Driver's License


In [92]:
browser.find_element(By.XPATH,"//input[contains(@id, 'pickupLocation')]").clear()
browser.find_element(By.XPATH,"//input[contains(@id, 'pickupLocation')]").click()
time.sleep(int(np.random.rand(1)*8))

browser.find_element(By.XPATH,"//input[contains(@id, 'pickupLocation')]").send_keys('Los Angeles')
time.sleep(int(np.random.rand(1)*8))
browser.find_element(By.XPATH,"//*[@id='c1-5-2-widget-pickupLocation']").send_keys(Keys.RETURN)
time.sleep(int(np.random.rand(1)*8))

#picks the driver's license
browser.find_element(By.CSS_SELECTOR, 'input.form-control.X-CountryOfResidence.AutoCompleteSelectInput').click()
time.sleep(int(np.random.rand(1)*8))
browser.find_element(By.XPATH, "//div[@class = 'autocomplete-suggestion'][@data-val = 'International']").click()

### Step 2: Enter pickup and drop off dates

In [93]:
#create a trip itinerary with start and end dates, trips are 17 days long because that's the average international booking
trip_day = date(2023, 1, 18)
trip = {}
trip['start_month'] = 'Jan'
trip['start_day'] = trip_day.strftime("%d")
trip['end_day'] = (trip_day + timedelta(17)).strftime("%d").lstrip('0')

In [94]:
#open the calendar
browser.find_element(By.XPATH, "//*[@id='c1-5-2-widget-pickupDate']").click()
time.sleep(int(np.random.rand(1)*7))
cal_month = browser.find_element(By.XPATH,"//div[@class = 'dr-cal-start']/div/div/div").find_element(By.CSS_SELECTOR,'button.dp-cal-month').text[0:3]

In [95]:
#Find the month where the trip starts
while cal_month != trip['start_month']:
    browser.find_element(By.XPATH, "//div[@class = 'dr-cal-end']/div/div/div/header/button[@class = 'dp-next']").click()
    #updates cal_month variable to whatever date you are currently on
    cal_month = browser.find_element(By.XPATH,"//div[@class = 'dr-cal-start']/div/div/div").find_element(By.CSS_SELECTOR,'button.dp-cal-month').text[0:3]
    time.sleep(int(np.random.rand(1)*3))
    print(cal_month)

Oct
Nov
Dec
Jan


In [96]:
#picks the start day of the trip
pu_d = browser.find_element(By.XPATH, f"//div[@class = 'dr-cal-start']/div/div/div/div/button[not(contains(@class, 'edge-day'))][text() = {trip['start_day']}]")
pu_d.click()

In [98]:
#picks the end day, if the start day is smaller than the end day the first page is picked, otherwise the next month's page is picked
if int(trip['start_day']) < int(trip['end_day']):
    do_d = browser.find_element(By.XPATH, f"//div[@class = 'dr-cal-start']/div/div/div/div/button[not(contains(@class, 'edge-day'))][text() = {trip['end_day']}]")
else:
    do_d = browser.find_element(By.XPATH, f"//div[@class = 'dr-cal-end']/div/div/div/div/button[not(contains(@class, 'edge-day'))][text() = {trip['end_day']}]")
do_d.click()

In [99]:
#click the search button
time.sleep(int(np.random.rand(1)*5))
browser.find_element(By.CSS_SELECTOR, 'button.btn.btn-success.btn-lg.btn-block.X-SearchButton').click()



### Step 3: Collect all the prices for a single trip in a single city

In [ ]:
    #"browser.execute_script(\"document.querySelector('#hbl-live-chat-wrapper').style.display = 'none';\")

    # scrollY = 0,
    # initialize an array1 = []\n",
    # while(you can scroll):\n",
    #    newentries = get the first batch of RVS (result.find_elements(By.CLASS_NAME, 'glider draggable'))\n",
    #    if array1 does not contain new entries then add newentries\n",
    #    scrollY = scrollY + 100\n",
    #    if scrollY >= viewport.height then get out of loop\n",
    # end while

In [102]:
#switch to correct window
browser.switch_to.window(browser.window_handles[1])

browser.execute_script("document.querySelector('#hbl-live-chat-wrapper').style.display = 'none';")
#used when debugging
#browser.refresh()

try:
    myElem = WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.CSS_SELECTOR,'div.campervan-result')))
    print ("Page is ready!")
except TimeoutException:
    print ("Loading took too much time!")

pixels = 0
webelements = []
data = []
for i in range(15):
    results = browser.find_elements(By.CLASS_NAME, 'campervan-result')
    print(pixels)
    time.sleep(1.5)
    browser.execute_script(f"window.scrollTo(0, {pixels});")

    for result in results:
        if result not in webelements:
            webelements.append(result)

            total_price = result.find_element(By.CLASS_NAME, 'X-VPrice-ConvertedFrom-Amount-Dollars').text
            daily_price = result.find_element(By.CSS_SELECTOR, 'div.quote-daily-price span.X-VPrice-ConvertedFrom-Amount-Dollars').text
            vehicle_class = result.find_element(By.CSS_SELECTOR, 'span.vehicle-name-text').text
            company_name = result.find_element(By.CSS_SELECTOR,'div.result-supplier img').get_attribute('alt')
            data.append({
                'total_price':total_price,
                'daily_price': daily_price,
                'vehicle_class':vehicle_class,
                'company_name':company_name
                })

    pixels += 500


print(webelements)
print(data)

pd.DataFrame(data)
    

Loading took too much time!
0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
[<selenium.webdriver.remote.webelement.WebElement (session="2744937ac5766f7de8733f50e0795513", element="1ae2b469-3b70-4c15-818c-044582bcde23")>, <selenium.webdriver.remote.webelement.WebElement (session="2744937ac5766f7de8733f50e0795513", element="15ecfdae-ef69-4acb-9a61-25e27d349766")>, <selenium.webdriver.remote.webelement.WebElement (session="2744937ac5766f7de8733f50e0795513", element="59d7993e-1ed8-4be0-9254-3a12a74b454d")>, <selenium.webdriver.remote.webelement.WebElement (session="2744937ac5766f7de8733f50e0795513", element="7397ea90-6027-4ac6-83ea-24b5d62a1c9f")>, <selenium.webdriver.remote.webelement.WebElement (session="2744937ac5766f7de8733f50e0795513", element="6d9b83e7-a537-4b7a-9f1f-7bf339246c04")>, <selenium.webdriver.remote.webelement.WebElement (session="2744937ac5766f7de8733f50e0795513", element="879ab90c-093a-4248-933a-305476cd0f03")>, <selenium.webdriver.remote.webelemen

,total_price,daily_price,vehicle_class,company_name
0,737,40,MiniVan,Travellers Autobarn USA
1,888,52,Wanderer,Lost Campers
2,952,52,Hitop Campervan,Travellers Autobarn USA
3,1025,56,Kuga Campervan,Travellers Autobarn USA
4,1072,63,Sierra,Lost Campers
5,1164,68,Hotel Sierra,Lost Campers
6,1311,77,Moab,Lost Campers
7,1402,82,Hotel Moab,Lost Campers
8,1508,88,Mavericks,Escape Campervans
9,1544,90,C19 Compact Motorhome,Cruise America


### Finding webelements to extract from div.campervan_result

In [53]:
# will have to tell script to wait for page to load when refactored to a loop
results = browser.find_elements(By.CLASS_NAME, 'campervan-result')

#total price
where_is_price = results[0].find_element(By.CLASS_NAME, 'X-VPrice-ConvertedFrom-Amount-Dollars').text


#Vehicle class
where_is_vehicle_name = results[0].find_element(By.CSS_SELECTOR, 'span.vehicle-name-text').text


#daily price
where_is_daily_price = results[0].find_element(By.CSS_SELECTOR, 'div.quote-daily-price span.X-VPrice-ConvertedFrom-Amount-Dollars').text

#company name
where_is_company_name = results[0].find_element(By.CSS_SELECTOR,'div.result-supplier img').get_attribute('alt')

print(where_is_daily_price, where_is_price, where_is_vehicle_name, where_is_company_name)

## Step 4: Itertate over all cities and all trip locations

In [ ]:
cities = [{'city':'San Francisco','state':'CA', 'abbr':'SFO', 'country':'USA'},
          {'city':'Las Vegas','state':'NV', 'abbr':'LAS', 'country':'USA'},
          {'city':'Los Angeles','state':'CA', 'abbr':'LAX', 'country':'USA'},
          {'city':'Seattle','state':'WA', 'abbr':'SEA', 'country':'USA'},
          {'city':'Salt Lake City','state':'UT', 'abbr':'SLC', 'country':'USA'},
          {'city':'Denver','state':'CO', 'abbr':'DEN', 'country':'USA'},
          {'city':'Chicago','state':'IL', 'abbr':'CHI', 'country':'USA'},
          {'city':'Dallas','state':'TX', 'abbr':'DFW', 'country':'USA'},
          {'city':'Orlando','state':'FL', 'abbr':'MCO', 'country':'USA'}]

In [47]:
#before looping through cities we will use a dictionary with only one city
city = {'city':'Los Angeles','state':'CA', 'abbr':'LAX', 'country':'USA'}

#javascript logic, not sure how this works, used to bypass a chat box?
click_chat_js = """
                    var button = document.querySelector('button.olark-launch-button.olark-size-md');
                    function eventFire(elem, etype) {
                      if (elem.fireEvent) {
                        elem.fireEvent('on' + etype);
                      } 
                      else {
                        var evObj = document.createEvent('Events');
                        evObj.initEvent(etype, true, false);
                        elem.dispatchEvent(evObj);
                      }
                    }
                    eventFire(button, 'click');
                """

deals_df = pd.DataFrame()

In [49]:
deals = []
for result in results:

    browser.execute_script("document.querySelector('#hbl-live-chat-wrapper').style.display = 'none';")
    
    if len(result.find_element(By.CSS_SELECTOR, 'span.X-VPrice-ConvertedFrom-Amount')) > 0:
    
        daily_rate = float(result.find_element(By.CSS_SELECTOR, 'span.X-VPrice-ConvertedFrom-Amount').text)
        pu_date = trip['start_date']
        do_date = trip['end_date']
        location = f'{city["city"]}, {city["state"]}'

        if browser.find_element(By.CSS_SELECTOR, 'div#olark-container').isDisplayed():
            browser.execute_script(click_chat_js)

        while True:
            try:
                result.find_by_text('Check Availability').click()
                break
            except ElementClickInterceptedException:
                try:
                    browser.execute_script("window.scrollTo(0, window.scrollY + 200);")
                except JavascriptException:
                    continue
        
        browser.switch_to.window(browser.window_handles[2])

        
        start = time.time()
        while True:
            try:
                if time.time() > start + 30:
                    browser.refresh()
                    # browser.windows.current.close()
                    # browser.windows.current = browser.windows[1]
                    # result.find_by_css('a.button-green.button-book').click()
                    # browser.windows.current = browser.windows[2]
                    start = time.time()
            except TimeoutException:
                continue
            try:
                company = browser.find_element(By.CSS_SELECTOR, 'img.vehicle-image')['src'].split('/')[-2].replace('-', ' ').lower().rstrip('us').strip().title()
                print(company)
                break
            except NoSuchElementException:
                continue
        rv_class = browser.find_element(By.CSS_SELECTOR, 'div.vehicle-name').text
        try:
            section = browser.find_element(By.CSS_SELECTOR,'span.X-VPrice-ConvertedFrom.v-price-total.v-price-converted-from').last
            total = float(section.find_element(By.CSS_SELECTOR, 'span.X-VPrice-ConvertedFrom-Amount').text)
            print(total)
        except NoSuchElementException:
            total = 'Not Listed'

        browser.close()
        browser.switch_to.window(browser.window_handles[1])
        

        deals.append({'Pickup Date':pu_date,'Dropoff Date':do_date,'Class':rv_class,'Company':company,
                    'Daily Rate':daily_rate,'Price':total,'Location':location})

        deals_df = deals_df.append(deals, ignore_index=True)

browser.close()

TypeError: 'WebElement' object is not iterable

In [108]:
from datetime import datetime

d1 = datetime.strptime("2022-08-30 17:54:00.000", "%Y-%m-%d %H:%M:%S.%f")
d2 = datetime.strptime("2022-08-30 10:24:00.000", "%Y-%m-%d %H:%M:%S.%f")

print(d1 - d2)

7:30:00


In [103]:
from datetime import datetime

d1 = datetime.strptime("2015-08-10 19:33:27.653", "%Y-%m-%d %H:%M:%S.%f")
d2 = datetime.strptime("2015-08-10 19:31:28.209", "%Y-%m-%d %H:%M:%S.%f")

print(d1 - d2)

datetime.datetime(2022, 8, 29, 15, 56, 47, 605995)

In [ ]:
cities = [{'city':'San Francisco','state':'CA', 'abbr':'SFO', 'country':'USA'},
          {'city':'Las Vegas','state':'NV', 'abbr':'LAS', 'country':'USA'},
          {'city':'Los Angeles','state':'CA', 'abbr':'LAX', 'country':'USA'},
          {'city':'Seattle','state':'WA', 'abbr':'SEA', 'country':'USA'},
          {'city':'Salt Lake City','state':'UT', 'abbr':'SLC', 'country':'USA'},
          {'city':'Denver','state':'CO', 'abbr':'DEN', 'country':'USA'},
          {'city':'Chicago','state':'IL', 'abbr':'CHI', 'country':'USA'},
          {'city':'Dallas','state':'TX', 'abbr':'DFW', 'country':'USA'},
          {'city':'Orlando','state':'FL', 'abbr':'MCO', 'country':'USA'}]

for city in cities:

    browser.find_element(By.XPATH,"//*[@id='c1-5-2-widget-pickupLocation']").click()
    time.sleep(3)    
  
    browser.find_element(By.XPATH,"//*[@id='c1-5-2-widget-pickupLocation']").send_keys(f"{city['city']}"+ " ")
    time.sleep(3)

    browser.find_element(By.XPATH,"//*[@id='c1-5-2-widget-pickupLocation']").send_keys(Keys.RETURN)